In [1]:
%run "/home/usriniva/uller_modified/discrete_time/Hanna_simu/simulation/impport_packages.ipynb"    #import all necessary packages - numpy, pandas etc
%run "/home/usriniva/uller_modified/discrete_time/Hanna_simu/simulation/simulation_class.ipynb"    #import all necessary packages - numpy, pandas etc
%run "/home/usriniva/uller_modified/discrete_time/Hanna_simu/simulation/create_world.ipynb"

### Reformulation

- Define the **maternal deviation**:


##### $\delta_t$= Phenotype of the mother - Genetic contribution of mother
##### $\delta_t$ = $P_{t-1} - G_{t-1}$

so that the phenotype is the sum of the genetic component of the current generation and maternal deviation from its phenotype:

$$
P = G_t + \omega * \delta_t + \varepsilon_{d}
$$

- We also separate mutation-related noise $\varepsilon_{m}$ from parental phenotype inheritance noise $\varepsilon_{epi}$.  
- We define a evolvable 'weight' term $\omega$: determines how much importance to give each term of the equation
- We also add developmental noise: purely environmental noise in teh form of  $\varepsilon_{d}$


The full new formulation becomes:

$$
Z' = (G_t + \varepsilon_{m}) + (\omega + \varepsilon_{epi})*(\delta_t) +  \varepsilon_{d}
$$

---

In [ ]:
def run_simulation(args):
    env_param_space, popsize, maxgen, dims, N, mutsize, mutrate, epsilon, nof_dimensions, only_genetics = args
    
    rho_m_alpha_beta = env_param_space[0:4]
    env_states = env_param_space[4:]

    meanw = np.zeros(maxgen)
    meanmemory_g = np.zeros(maxgen)
    meanmemory_p= np.zeros(maxgen)
    meanneutral_g = np.zeros(maxgen)
    meanneutral_p = np.zeros(maxgen)
    
    for t in range(0, (maxgen-1)):
        #if t%10 == 0:
        #    print(t)
        for d in range(0, nof_dimensions):
            N[:,dims['dev'][d]] = N[:,dims['pheno'][d]] - env_states[t]
            
        dev_combined = np.sqrt(np.sum(N[:,dims['dev']]**2, axis =1))     
        W = np.exp((-dev_combined**2)/ (2))
        
        
        meanw[t] = np.mean(W)    
        
        meanmemory_g[t] = np.mean(N[:, dims['memory']])  # the actual memory
        meanmemory_p[t] = np.mean(1/(1 + np.exp(-N[:,dims['memory']]))) #with the logistc correction
        
        meanneutral_g[t] = np.mean(N[:,dims['neutral']])
        meanneutral_p[t] = np.mean(1/(1 + np.exp(-N[:,dims['neutral']])))
        
        #current gen offspring (the cube gets stored here per gen)
        
        offspring = np.zeros((popsize, dims['total_layers'])) # empty 3D matrix with dimensions of the cube
        
        #sample offspring for each scenario weighted by fitness
        
        
        parents_idx = np.random.choice(popsize, size=popsize, p = (W/np.sum(W)) )#pick #popsize sized random numbers
        offspring[:,:] = N[parents_idx, :] #similar to how it works in matlab
        
        mutate_memories = np.random.uniform(low=0, high=1, size= popsize) < mutrate[0]
        #mutate_neutral = np.random.uniform(low=0, high=1, size=popsize) < mutrate[0]
        mutate_geno = np.random.uniform(low=0, high=1, size=(popsize, nof_dimensions)) < mutrate[1]
        
        # adding a mutation of size mutsize[1], rate mutate_memories, to the epi-memory
        # the epignetic memory evolves; 
        
        offspring[:,dims['memory']] = (
                                offspring[:,dims['memory']] 
                                + mutate_memories * np.random.normal(0, mutsize[0], popsize)
        )
        
        # adding a mutation of size mutsize[1] (epi-mutation size), rate mutate_memories, to the neutral phenotype
        # the neutral trait recieves random mutation but is not implemented in the calculation for fitness; so is evolving neutrally
        offspring[:,dims['neutral']] = (
                                offspring[:,dims['neutral']]  
                                + mutate_memories * np.random.normal(0, mutsize[0], popsize)
        ) 
        
        # adding a mutation of size mutsize[2] (genetic mutation size), rate mutate_memories, to the genotype

        offspring[:,dims['geno']] = (
                            offspring[:,dims['geno']] 
                            + mutate_geno * np.random.normal(0, mutsize[1], size=(popsize, nof_dimensions) )
        )
        
        if only_genetics :
            if t % 1000 == 0:
                print(f"Gen {t}: Running ONLY genetics mode")
            
            for i in range(0, nof_dimensions):
                offspring[:, dims['pheno'][i]] = (
                offspring[:, dims['geno'][i]]
                + np.random.normal(0, epsilon, size=popsize) )
        else:
            if t % 1000 == 0:
                print(f"Gen {t}: Running EPIGENETIC mode")
            
            for i in range(0, nof_dimensions):
                    offspring[:, dims['pheno'][i]] = (
                        offspring[:, dims['geno'][i]] #genotype
                        + np.random.normal(0, epsilon, size=popsize) # adding random noise (developemental noise, nromally distributed with epsilon variance)
                        + ((1 / (1 + np.exp(-offspring[:, dims['memory']]))) #phenotype version of the memory (y axis) 
                        * (N[parents_idx, dims['pheno'][i]] - N[parents_idx, dims['geno'][i]]))# adding the deviation -> read as weight (phenotypic component * deviation)
                    )
        N = offspring
        
        
    return {                ## Make sure this is indented!!
        'maxgen_popsize': [maxgen,popsize],
        'rho_m_alpha_beta' :rho_m_alpha_beta,
        'meanw': meanw,
        'meanmemory_g': meanmemory_g,
        'meanmemory_p': meanmemory_p,
        'meanneutral_g': meanneutral_g,
        'meanneutral_p': meanneutral_p
        }
        
        

In [6]:
rho_cases = 20
m_cases= 21
maxgen= 1000


full_env_values = create_world(rho_cases, m_cases, maxgen)   # create world (input: generations, no. rhos and no. ms) 
print(full_env_values.shape)

clean_data_env = full_env_values[~np.isnan(full_env_values).any(axis=1)]

print('Total No. of rhos:',len(np.unique([full_env_values[:,0]])))
print('Total No. of valid rhos:', len(np.unique([clean_data_env[:,0]])))
nof_scenarios = np.shape(clean_data_env)[0]
print('Total no of valid rho-m combinations i.e. nof_scenarios:', nof_scenarios)



(420, 1004)
Total No. of rhos: 20
Total No. of valid rhos: 20
Total no of valid rho-m combinations i.e. nof_scenarios: 283


In [ ]:
#extract environmental parameters and environemntal optima into seperate objects


rho_m_alpha_beta = clean_data_env[0:4]
env_states = clean_data_env[4:]


nof_scenarios: 283


In [5]:
popsize = 1000

# Mutation rates: [epi-memory mutation rate, geno mutation rate]
mutrate = [0.01, 0.01]

# Mutation sizes: [epi-memory mutation size, geno mutation size]
mutsize = [1, 1]

epsilon = 0.05 # phenotypic noise

# Number of dimensions (can be 1 or more)
nof_dimensions = 2
                                 
initial_memory = -3


In [ ]:
# get the cube dimensions
dims = get_cube_dims(nof_dimensions)

#create the cube
N = create_cube(popsize, dims=dims, nof_dimensions=nof_dimensions, epsilon=epsilon, initial_memory=initial_memory)

print(N.shape)



if __name__ == '__main__':
    
    # Create all combinations of sigma_mut and sigma_alpha
    param_grid = [
        (scenario, popsize, maxgen, dims, N, mutsize, mutrate, epsilon, nof_dimensions, True) #only genetics can either be True or false
        for _, scenario in enumerate(clean_data_env) #where each row corresponds to one scenario
    ]

    num_cpus = 10
    
    with concurrent.futures.ProcessPoolExecutor(num_cpus) as executor: 
        results_array = list(executor.map(run_simulation, param_grid))


(1000, 8)
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics modeGen 0: Running ONLY genetics modeGen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode


Gen 0: Running ONLY genetics modeGen 0: Running ONLY genetics modeGen 0: Running ONLY genetics mode

Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode

Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 0: Running ONLY genetics mode
Gen 

In [ ]:
# Get all keys 
keys = results_array[0].keys()

# Concatenate each key's arrays vertically
concatenated_results = {}
for k in keys:
    concatenated_results[k] = np.vstack([r[k] for r in results_array])

{'maxgen_popsize': array([[1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000, 1000],
        [1000,

In [243]:
path= '/home/usriniva/uller_modified/discrete_time/Hanna_simu/results/only_gen'  #neutral folder
#path= '/home/usriniva/uller_modified/discrete_time/Hanna_simu/results/epi'   #epi folder
#------ Different starting point ------

#path = '/home/usriniva/uller_modified/discrete_time/Hanna_simu/results/startingmem_0/only_gen'
#path = '/home/usriniva/uller_modified/discrete_time/Hanna_simu/results/startingmem_0/epi'

#-----long time------
#path= '/home/usriniva/uller_modified/discrete_time/Hanna_simu/results/long_time'

filename=f'/results_dim{nof_dimensions}_og.pkl'
#filename=f'/results_dim{nof_dimensions}.pkl'

#-----

#path='/home/usriniva/uller_modified/discrete_time/Hanna_simu/results/startingmem_0_hi_ss/epi'
#path='/home/usriniva/uller_modified/discrete_time/Hanna_simu/results/startingmem_0_hi_ss/only_gen'

with open(f'{path}/{filename}', 'wb') as f:
    pickle.dump(concatenated_results, f)